<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/cut_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/VictorUceda/TrackNet-Padel

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/VictorUceda/TrackNet-Padel
   8321e08..0ebc4c4  main       -> origin/main
Updating 8321e08..0ebc4c4
Fast-forward
 cut_fragments.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

video_path = '/content/gdrive/MyDrive/PadelMatchs/WPTDanish_Octavos_PD_0_01.mp4'
points_path = video_path[:-4]+'.points.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd TrackNet-Padel

/content/TrackNet-Padel


In [ ]:
import pandas as pd

df_segments = pd.read_csv(points_path, sep=';')
i = 0
for index, row in df_segments.iterrows():
  name = video_path[:-4]+'_'+str(i)+'.mp4'
  print(row['ini'], row['fin'], name)
  !ffmpeg -ss {row['ini']/25} -i {video_path} -c copy -t {(row['ini']-row['fin'])/25} {name}
  i += 1